In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install conda

Note: you may need to restart the kernel to use updated packages.


In [3]:
conda activate table-understanding


CondaError: Run 'conda init' before 'conda activate'


Note: you may need to restart the kernel to use updated packages.


In [4]:
conda list openpyxl

# packages in environment at /Users/dhyeydesai/miniforge3:
#
# Name                    Version                   Build  Channel
openpyxl                  3.1.5           py310h0e452f4_1    conda-forge

Note: you may need to restart the kernel to use updated packages.


In [5]:
import sys
!conda install --yes --prefix {sys.prefix} openpyxl

!conda activate table-understanding

Channels:
 - conda-forge
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


CondaError: Run 'conda init' before 'conda activate'



In [6]:
import pandas

import json
import requests
import os
import io
import subprocess
import yaml
from typing import List, Tuple, Optional
import zipfile
import sys
import platform
import traceback
import pandas as pd
from conda.cli.python_api import run_command, Commands
from conda.exceptions import EnvironmentLocationNotFound

/var/folders/8z/99p9qj_s35zd5p2_p1g6xh4w0000gn/T/ipykernel_97652/839853245.py:15: DeprecationWarning: conda.cli.python_api is deprecated and will be removed in 24.9. Use `conda.testing.conda_cli` instead.
  from conda.cli.python_api import run_command, Commands


In [7]:
def download_file(url: str, filename: str):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            f.write(response.content)
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")
        raise

In [8]:
def load_dataset_info(json_file_path: str) -> Tuple[float, float, List[str], List[List[str]]]:
    try:
        with open(json_file_path, 'r') as file:
            data = json.load(file)

        study = data['study'][0]

        lat = study['site'][0]['geo']['geometry']['coordinates'][1]
        lon = study['site'][0]['geo']['geometry']['coordinates'][0]

        file_urls = []
        variables_list = []

        for site in study['site']:
            for paleo_data in site['paleoData']:
                # Access the first element within 'dataFile'
                data_file = paleo_data['dataFile'][0]

                file_urls.append(data_file['fileUrl'])

                # Access 'variables' within the first element of 'dataFile'
                variables_list.append([v.get('variableName', None) for v in data_file.get('variables', [])])

        return lat, lon, file_urls, variables_list

    except (FileNotFoundError, json.JSONDecodeError, KeyError) as e:
        print(f"Error loading dataset info: {e}")
        raise

In [9]:
import requests
import io

def download_and_process_data(file_url: str, variables: Optional[list[str]] = None) -> pandas.DataFrame:
    try:
        response = requests.get(file_url)
        response.raise_for_status()
        content = response.text

        # Find the start of the data section
        data_start = content.find('Data:')
        if data_start == -1:
            raise ValueError("Data section not found in the file")

        # Extract the data section
        data_section = content[data_start + 5:].strip()  # Skip 'Data:' and any leading/trailing whitespace

        # Split the data section into lines
        data_lines = data_section.splitlines()

        # Find the first non-comment line (assuming it contains the header)
        header_line = next((line for line in data_lines if not line.startswith('#')), None)

        if header_line:
            # Remove comment character if present and split into column names
            column_names = [col.replace('#', '').strip() for col in header_line.split()]
            # Filter out empty column names
            column_names = [col for col in column_names if col]

            # Read the data, skipping the first row (header) if it's duplicated
            df = pandas.read_csv(io.StringIO('\n'.join(data_lines)), delim_whitespace=True, comment='#', header=0, names=column_names)
            if df.iloc[0].equals(pandas.Series(column_names)):  # Check if first row is the header
                df = df.iloc[1:].copy()  # Skip the first row if it's a duplicate header
        else:
            # If no header line is found, read the data without column names
            df = pandas.read_csv(io.StringIO(data_section), delim_whitespace=True, comment='#', header=None)

        if df.empty:
            raise ValueError("No data was parsed from the file")

        # If variables are provided and match the number of columns, use them
        if variables and len(variables) == df.shape[1]:
            df.columns = variables

        print(f"Successfully processed data from {file_url}. Shape: {df.shape}")
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error downloading data from {file_url}: {e}")
    except ValueError as e:
        print(f"Error processing data from {file_url}: {e}")
    except Exception as e:
        print(f"Unexpected error processing data from {file_url}: {e}")

    return pandas.DataFrame()

# Example usage:
urls = [
    "https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dust-coreb.txt",
    "https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dmar-ens.txt",
    "https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dmar.txt",
    "https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dust-corea.txt"
]

for url in urls:
    print(f"Processing file: {url}")
    # You can provide variables if you know them, or leave it as None
    variables = None  # or [list of variable names if known]
    df = download_and_process_data(url, variables)
    if not df.empty:
        print(df.head())
        print(f"Columns: {df.columns.tolist()}")
    print("\n")

Processing file: https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dust-coreb.txt


/var/folders/8z/99p9qj_s35zd5p2_p1g6xh4w0000gn/T/ipykernel_97652/3268293368.py:31: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pandas.read_csv(io.StringIO('\n'.join(data_lines)), delim_whitespace=True, comment='#', header=0, names=column_names)


Successfully processed data from https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dust-coreb.txt. Shape: (4341, 4)
   age_calBP  dustfrac   sedrate      dens
0    4483.51  0.168678  0.055556  0.195990
1    4482.57  0.156434  0.055556  0.200526
2    4481.46  0.164476  0.055556  0.205818
3    4480.36  0.225408  0.055556  0.211110
4    4479.26  0.131894  0.055556  0.216402
Columns: ['age_calBP', 'dustfrac', 'sedrate', 'dens']


Processing file: https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dmar-ens.txt


/var/folders/8z/99p9qj_s35zd5p2_p1g6xh4w0000gn/T/ipykernel_97652/3268293368.py:31: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pandas.read_csv(io.StringIO('\n'.join(data_lines)), delim_whitespace=True, comment='#', header=0, names=column_names)


Successfully processed data from https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dmar-ens.txt. Shape: (199, 6)
   age_calBP     DMAR   CI_0.84  CI_0.16  CI_0.25  CI_0.975
0     -46.55  40.1265   55.9888  24.8088  -2.3479   72.0817
1     -23.64  41.6070   59.0063  23.8030  15.7566   85.1572
2      -0.74  74.0322  100.2443  46.9366  32.8553  135.4476
3      22.17  50.7744   77.1108  27.8263  17.7682  103.2618
4      45.07  21.4330   29.8379  11.7334   7.7101   42.9134
Columns: ['age_calBP', 'DMAR', 'CI_0.84', 'CI_0.16', 'CI_0.25', 'CI_0.975']


Processing file: https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dmar.txt


/var/folders/8z/99p9qj_s35zd5p2_p1g6xh4w0000gn/T/ipykernel_97652/3268293368.py:31: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pandas.read_csv(io.StringIO('\n'.join(data_lines)), delim_whitespace=True, comment='#', header=0, names=column_names)


Successfully processed data from https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dmar.txt. Shape: (2471, 2)
   age_calBP     DMAR
0     -60.12  58.8432
1     -58.75  71.7891
2     -57.38  44.6904
3     -56.01  36.6339
4     -54.65  38.6674
Columns: ['age_calBP', 'DMAR']


Processing file: https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dust-corea.txt
Successfully processed data from https://www.ncei.noaa.gov/pub/data/paleo/paleolimnology/northamerica/usa/colorado/blue2019dust-corea.txt. Shape: (2769, 4)
   age_calBP  dustfrac   sedrate     dens
0    2722.83  0.337020  0.049261  0.24186
1    2722.12  0.491964  0.049261  0.24114
2    2721.21  0.272375  0.049261  0.24424
3    2720.30  0.262410  0.049261  0.24734
4    2719.48  0.272334  0.049261  0.25009
Columns: ['age_calBP', 'dustfrac', 'sedrate', 'dens']




/var/folders/8z/99p9qj_s35zd5p2_p1g6xh4w0000gn/T/ipykernel_97652/3268293368.py:31: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pandas.read_csv(io.StringIO('\n'.join(data_lines)), delim_whitespace=True, comment='#', header=0, names=column_names)


In [10]:
import subprocess

def setup_conda_environment():
    env_name = "table-understanding"

    # Create the environment
    create_env_cmd = ["conda", "create", "-n", env_name, "python=3.10", "-y"]
    result = subprocess.run(create_env_cmd, capture_output=True, text=True)

    if result.returncode == 0:
        print("Environment created successfully.")
    else:
        print(f"Error creating environment: {result.stderr}")
        return

    # Install packages
    install_cmd = ["conda", "run", "-n", env_name, "pip", "install", "pandas", "requests", "pyyaml"]
    result = subprocess.run(install_cmd, capture_output=True, text=True)

    if result.returncode == 0:
        print("Packages installed successfully.")
    else:
        print(f"Error installing packages: {result.stderr}")

setup_conda_environment()

Environment created successfully.
Packages installed successfully.


In [11]:
def run_in_conda_env(env_name: str, command: str):
    try:
        result = run_command(Commands.RUN, ["-n", env_name] + command.split())
        return result
    except Exception as e:
        print(f"Error running command in conda environment: {e}")
        raise

In [12]:
def setup_isi_table_understanding(env_name: str, skip_psl_download: bool = False):
    try:
        if os.path.exists("isi-table-understanding"):
            print("isi-table-understanding directory already exists. Updating...")
            os.chdir("isi-table-understanding")
            subprocess.run(["git", "pull"], check=True)
        else:
            subprocess.run(["git", "clone", "-b", "impl", "https://github.com/usc-isi-i2/isi-table-understanding.git"], check=True)
            os.chdir("isi-table-understanding")
        if os.path.exists("requirements.txt"):
            run_in_conda_env(env_name, "pip install -r requirements.txt")
        else:
            print("requirements.txt not found. Skipping package installation.")
        if not skip_psl_download:
            os.makedirs("data", exist_ok=True)
            psl_url = input("Please enter the URL for the PSL files zip (or press Enter to skip): ")
            if psl_url:
                try:
                    download_file(psl_url, "data/psl_files.zip")
                    with zipfile.ZipFile("data/psl_files.zip", 'r') as zip_ref:
                        zip_ref.extractall("data/")
                    print("PSL files downloaded and extracted successfully.")
                except requests.exceptions.RequestException as e:
                    print(f"Failed to download PSL files: {e}")
                    print("Continuing without PSL files. Some functionality may be limited.")
            else:
                print("Skipping PSL files download.")
        else:
            print("Skipping PSL files download.")
        if not os.path.exists("InferSent"):
            subprocess.run(["git", "clone", "https://github.com/facebookresearch/InferSent.git"], check=True)
        os.chdir("InferSent")
        os.makedirs("GloVe", exist_ok=True)
        glove_url = "http://nlp.stanford.edu/data/glove.840B.300d.zip"
        download_file(glove_url, "GloVe/glove.840B.300d.zip")
        with zipfile.ZipFile("GloVe/glove.840B.300d.zip", 'r') as zip_ref:
            zip_ref.extractall("GloVe/")
        os.makedirs("encoder", exist_ok=True)
        download_file("https://dl.fbaipublicfiles.com/infersent/infersent1.pkl", "encoder/infersent1.pkl")
        download_file("https://dl.fbaipublicfiles.com/infersent/infersent2.pkl", "encoder/infersent2.pkl")
        os.chdir("..")
        psl_config = {
            "psl": {
                "model_path": os.path.abspath("path/to/pretrained/psl/model"),
                "infersent_path": os.path.abspath("InferSent"),
                "glove_path": os.path.abspath("InferSent/GloVe/glove.840B.300d.txt")
            }
        }
        os.makedirs("cfg", exist_ok=True)
        with open("cfg/psl_config.yaml", "w") as f:
            yaml.dump(psl_config, f)
        print("ISI Table Understanding setup completed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error in setup: {e}")
        raise
    except Exception as e:
        print(f"Unexpected error in setup: {e}")
        raise

In [13]:
def run_table_understanding(env_name: str, input_file: str, output_dir: str):
    try:
        config_file = "cfg/psl_config.yaml"
        files_config = f"- {input_file}"

        with open("cfg/files.yaml", "w") as f:
            f.write(files_config)

        result = run_in_conda_env(env_name, f"python main.py --config {config_file} --files cfg/files.yaml --output {output_dir}")
        return result
    except Exception as e:
        print(f"Error running table understanding: {e}")
        raise

In [14]:
def main(skip_psl_download: bool = False):
    try:
        json_file_path = 'data.json'
        output_dir = os.path.abspath('./output')

        print(f"Current working directory: {os.getcwd()}")
        print(f"Attempting to create output directory: {output_dir}")

        # Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)
        print(f"Output directory created/confirmed: {output_dir}")
        print(f"Output directory exists: {os.path.exists(output_dir)}")
        print(f"Output directory is writable: {os.access(output_dir, os.W_OK)}")

        lat, lon, file_urls, variables_list = load_dataset_info(json_file_path)

        print(f"Latitude: {lat}, Longitude: {lon}")

        env_name = setup_conda_environment()
        setup_isi_table_understanding(env_name, skip_psl_download)

        results = []
        for file_url, variables in zip(file_urls, variables_list):
            print(f"Processing file: {file_url}")
            print(f"Variables: {variables}")

            # If variables are all None, set it to None to let download_and_process_data handle it
            if all(var is None for var in variables):
                variables = None

            df = download_and_process_data(file_url, variables)
            if df.empty:
                print(f"Skipping empty dataframe for {file_url}")
                continue

            print("DataFrame head:")
            print(df.head())

            csv_file = os.path.join(output_dir, f"{os.path.basename(file_url)}.csv")
            print(f"Attempting to save file: {csv_file}")
            try:
                df.to_csv(csv_file, index=False)
                print(f"Saved data to {csv_file}")
            except Exception as e:
                print(f"Error saving data to {csv_file}: {e}")
                print(f"File path exists: {os.path.exists(os.path.dirname(csv_file))}")
                print(f"File path is writable: {os.access(os.path.dirname(csv_file), os.W_OK)}")
                continue

            try:
                table_understanding_result = run_table_understanding(env_name, csv_file, output_dir)

                # Load table understanding output
                with open(os.path.join(output_dir, "table-understanding-results.json"), "r") as f:
                    table_blocks = json.load(f)

                # Evaluate block identification
                for block in table_blocks:
                    block_columns = block["col_headers"]
                    block_data = block["cells"]

                    # Check if all columns in a block correspond to a single variable
                    is_correct = any(all(col in var for col in block_columns) for var in variables)

                    print(f"Block: {block_columns}")
                    print(f"Data Series:")
                    for row in block_data:
                        print(row)
                    print(f"Correctly identified: {is_correct}\n")

            except Exception as e:
                print(f"Error running table understanding or evaluating results: {e}")
                table_understanding_result = None

            results.append({
                'file_url': file_url,
                'dataframe': df,
                'table_understanding_result': table_understanding_result
            })

            print("\n")

        print("Processing complete. Check the output directory for results.")
        return results

    except Exception as e:
        print(f"An error occurred in main: {e}")
        traceback.print_exc()
        return None


In [ ]:
# Run the main function and display results
results = main(skip_psl_download=True)
if results:
    for result in results:
        print(f"File: {result['file_url']}")
        print("DataFrame head:")
        display(result['dataframe'].head())
        print("Table Understanding Result:")
        print(result['table_understanding_result'])
        print("\n")
else:
    print("No results were returned. Check the error messages above for more information.")

Current working directory: /Users/dhyeydesai/Downloads/unique
Attempting to create output directory: /Users/dhyeydesai/Downloads/unique/output
Output directory created/confirmed: /Users/dhyeydesai/Downloads/unique/output
Output directory exists: True
Output directory is writable: True
Latitude: 119.078, Longitude: 1.4033
Environment created successfully.
Packages installed successfully.


Cloning into 'isi-table-understanding'...


requirements.txt not found. Skipping package installation.
Skipping PSL files download.


Cloning into 'InferSent'...
